<a href="https://colab.research.google.com/github/jxxngho/HG_MLDL/blob/main/04_1_Logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 럭키백의 확률

In [ ]:
# 럭키백에 들어간 생선의 크기, 무게 등이 주어졌을 때 7개 생선에 대한 확률을 출력해야한다!
# --> k-최근접 이웃은 주변 이웃을 찾아주니까 이웃의 클래스 비율을 확률이라고 출력하면 되지 않을까?

# 데이터 준비

In [2]:
import pandas as pd
fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [3]:
print(pd.unique(fish['Species']))

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


In [5]:
# 입력 데이터로 사용
fish_input = fish[['Weight','Length','Diagonal','Height','Width']].to_numpy()
print(fish_input[:5])

[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]
 [363.      29.      33.5     12.73     4.4555]
 [430.      29.      34.      12.444    5.134 ]]


In [6]:
# 타깃 데이터
fish_target = fish['Species'].to_numpy()

In [7]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(fish_input,fish_target, random_state=42)

In [8]:
# 표준화 전처리
# 훈련 세트의 통계 값으로 테스트 세트를 변환해야 한다!!
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

# k-최근접 이웃 분류기의 확률 예측

In [10]:
from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier(n_neighbors=3)

kn.fit(train_scaled, train_target)
print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))

# 타깃 데이터에 2개 이상의 클래스가 포함된 문제를 다중분류라고 부른다.
# 다중 분류에서도 타깃값을 숫자로 바꾸어 입력할 수 있지만 사이킷런에서는 문자열로 된 타깃값을 그대로 사용가능
# 주의할 점은 !  타깃값을 그대로 사이킷런 모델에 전달하면 순서가 자동으로 알파벳 순으로 매겨짐
# 따라서 pd.unique(fish['Species])로 출력했던 순서와 다르다

0.8907563025210085
0.85


In [11]:
# KNeighborsClassfier에서 정렬된 타깃값은 classes_ 속성에 저장되어 있다.
print(kn.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [13]:
print(kn.predict(test_scaled[:5]))
# 이 5개 샘플에 대한 예측은 어떤 확률로 만들어졌을까?
# --> predict_proba() 메서드로 클래스별 확률값을 반환

['Perch' 'Smelt' 'Pike' 'Perch' 'Perch']


In [14]:
import numpy as np
proba = kn.predict_proba(test_scaled[:5])
print(np.round(proba,decimals=4))

# 이 모델이 계산한 확률이 가장 가까운 이웃의 비율이 맞는지 확인해보자

[[0.     0.     1.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     1.     0.    ]
 [0.     0.     0.     1.     0.     0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]]


In [18]:
distances, indexes = kn.kneighbors(test_scaled[3:4])
print(train_target[indexes])

# 이 샘플의 이웃은 다섯 번째 클래스인 'Roach'가 1개, 세 번째 클래스인 'Perch'가 2개
# 따라서 다섯 번째 클래스에 대한 확률은 1/3 이고 세 번째 클래스에 대한 확률은 2/3 = 0.6667이 된다

# 이상한 점 ? -> 3개의 최근접 이웃을 사용하기 때문에 가능한 확률은 0/3, 1/3, 2/3 3/3 뿐이다....

[['Roach' 'Perch' 'Perch']]
